<a href="https://colab.research.google.com/github/sof1a03/KDE-group6/blob/main/ISBNcorrection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from multiprocessing.pool import ThreadPool

In [ ]:
books = pd.read_csv("Books.csv")

<ipython-input-2-f46b15fa6211>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("Books.csv")


In [ ]:
books['ISBN'] = books['ISBN'].str.upper()

In [ ]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book_Title           271360 non-null  object
 2   Book_Author          271358 non-null  object
 3   Year_Of_Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image_URL_S          271360 non-null  object
 6   Image_URL_M          271360 non-null  object
 7   Image_URL_L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install graphframes
!pip install
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 123634 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u432-ga~us1-0ubuntu2~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u432-ga~us1-0ub

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col,
    udf,
    row_number,
    countDistinct,
    collect_list,
    struct,
    count,
    sum,
    avg,
    expr,
    lit,
    percentile_approx,
    max as spark_max,
    explode,
    least,
    greatest
)
from pyspark.sql.types import StringType, IntegerType, BinaryType, DoubleType, ArrayType, StructType, StructField
from pyspark.sql import Window
from datetime import datetime
from graphframes import GraphFrame
from scipy.sparse import csr_matrix, vstack, hstack
import numpy as np
import pickle
import base64
import pandas as pd


In [ ]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("ISBNValidationandAPICheck") \
    .master("local[*]") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.1-s_2.12") \
    .config("spark.executor.memory", "20G") \
    .config("spark.driver.memory", "50G") \
    .config("spark.executor.memoryOverhead", "1G") \
    .config("spark.default.parallelism", "100") \
    .config("spark.sql.shuffle.partitions", "10") \
    .config("spark.driver.maxResultSize", "2G") \
    .getOrCreate()



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length, when, expr, lit, sum as spark_sum

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Corrected ISBN Validation Debugged") \
    .getOrCreate()

# Load the dataset
books = spark.read.csv("Books.csv", header=True, inferSchema=True)

# Normalize ISBN column: Remove hyphens and whitespace
books = books.withColumn("ISBN", expr("regexp_replace(trim(ISBN), '-', '')"))

# Validate ISBN Length (10 or 13)
books = books.withColumn(
    "is_valid_length",
    when((length(col("ISBN")) == 10) | (length(col("ISBN")) == 13), True).otherwise(False)
)

# Add position column (POS) for each character in ISBN
books_with_pos = books.withColumn("POS", expr("sequence(1, length(ISBN))"))
books_exploded = books_with_pos.withColumn("POS", expr("explode(POS)"))

# Extract character values using substring
books_exploded = books_exploded.withColumn(
    "digit_value",
    when(expr("substring(ISBN, POS, 1)") == "X", lit(10))
    .otherwise(expr("CAST(substring(ISBN, POS, 1) AS INT)"))
)

# Add weights for ISBN-10 (Corrected Formula)
books_exploded = books_exploded.withColumn(
    "weight_10",
    when(length("ISBN") == 10, 11 - col("POS")).otherwise(0)
)

# Add weights for ISBN-13
books_exploded = books_exploded.withColumn(
    "weight_13",
    when(length("ISBN") == 13, when((col("POS") % 2) == 1, 1).otherwise(3)).otherwise(0)
)

# Compute weighted contributions for ISBN-10 and ISBN-13
books_exploded = books_exploded.withColumn(
    "weighted_value_10",
    col("digit_value") * col("weight_10")
).withColumn(
    "weighted_value_13",
    col("digit_value") * col("weight_13")
)

# Sum contributions for ISBN-10 and ISBN-13
weighted_sum = books_exploded.groupBy("ISBN").agg(
    spark_sum("weighted_value_10").alias("weighted_sum_10"),
    spark_sum("weighted_value_13").alias("weighted_sum_13"),
    spark_sum(
        when(col("POS") == 10, col("digit_value")).otherwise(0)
    ).alias("last_digit")
)

# Add checksum validations for ISBN-10 and ISBN-13
weighted_sum = weighted_sum.withColumn(
    "checksum_valid_10",
    (length(col("ISBN")) == 10) & (((col("weighted_sum_10")) % 11) == 0)
).withColumn(
    "checksum_valid_13",
    (length(col("ISBN")) == 13) & ((col("weighted_sum_13") % 10) == 0)
)

# Combine checksum validations
weighted_sum = weighted_sum.withColumn(
    "Validity",
    when(col("checksum_valid_10") | col("checksum_valid_13"), "Valid").otherwise("Invalid")
)

# Debugging: Show intermediate results for ISBN 0671002481
debug_weights = books_exploded.filter(col("ISBN") == "0671002481")
debug_weights.select("ISBN", "POS", "digit_value", "weight_10", "weighted_value_10").show(truncate=False)

debug_sum = weighted_sum.filter(col("ISBN") == "0671002481")
debug_sum.select("ISBN", "weighted_sum_10", "last_digit", "checksum_valid_10", "Validity").show(truncate=False)

# Show all ISBNs with their validity status
weighted_sum.select("ISBN", "Validity").show(truncate=False)

# Stop Spark Session
spark.stop()


+----------+---+-----------+---------+-----------------+
|ISBN      |POS|digit_value|weight_10|weighted_value_10|
+----------+---+-----------+---------+-----------------+
|0671002481|1  |0          |10       |0                |
|0671002481|2  |6          |9        |54               |
|0671002481|3  |7          |8        |56               |
|0671002481|4  |1          |7        |7                |
|0671002481|5  |0          |6        |0                |
|0671002481|6  |0          |5        |0                |
|0671002481|7  |2          |4        |8                |
|0671002481|8  |4          |3        |12               |
|0671002481|9  |8          |2        |16               |
|0671002481|10 |1          |1        |1                |
+----------+---+-----------+---------+-----------------+

+----------+---------------+----------+-----------------+--------+
|ISBN      |weighted_sum_10|last_digit|checksum_valid_10|Validity|
+----------+---------------+----------+-----------------+--------+


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length, when, expr, lit, sum as spark_sum

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Corrected ISBN Validation Debugged") \
    .getOrCreate()

# Load the dataset
books = spark.read.csv("Books.csv", header=True, inferSchema=True)

# Normalize ISBN column: Remove hyphens and whitespace
books = books.withColumn("ISBN", expr("regexp_replace(trim(ISBN), '-', '')"))

# Validate ISBN Length (10 or 13)
books = books.withColumn(
    "is_valid_length",
    when((length(col("ISBN")) == 10) | (length(col("ISBN")) == 13), True).otherwise(False)
)

# Add position column (POS) for each character in ISBN
books_with_pos = books.withColumn("POS", expr("sequence(1, length(ISBN))"))
books_exploded = books_with_pos.withColumn("POS", expr("explode(POS)"))

# Extract character values using substring
books_exploded = books_exploded.withColumn(
    "digit_value",
    when(expr("upper(substring(ISBN, POS, 1))") == "X", lit(10))
    .otherwise(expr("CAST(substring(ISBN, POS, 1) AS INT)"))
)

# Add weights for ISBN-10 (Corrected Formula)
books_exploded = books_exploded.withColumn(
    "weight_10",
    when(length("ISBN") == 10, 11 - col("POS")).otherwise(0)
)

# Add weights for ISBN-13
books_exploded = books_exploded.withColumn(
    "weight_13",
    when(length("ISBN") == 13, when((col("POS") % 2) == 1, 1).otherwise(3)).otherwise(0)
)

# Compute weighted contributions for ISBN-10 and ISBN-13
books_exploded = books_exploded.withColumn(
    "weighted_value_10",
    col("digit_value") * col("weight_10")
).withColumn(
    "weighted_value_13",
    col("digit_value") * col("weight_13")
)

# Sum contributions for ISBN-10 and ISBN-13
weighted_sum = books_exploded.groupBy("ISBN").agg(
    spark_sum("weighted_value_10").alias("weighted_sum_10"),
    spark_sum("weighted_value_13").alias("weighted_sum_13"),
    spark_sum(
        when(col("POS") == 10, col("digit_value")).otherwise(0)
    ).alias("last_digit")
)

# Add checksum validations for ISBN-10 and ISBN-13
weighted_sum = weighted_sum.withColumn(
    "checksum_valid_10",
    (length(col("ISBN")) == 10) & (((col("weighted_sum_10")) % 11) == 0)
).withColumn(
    "checksum_valid_13",
    (length(col("ISBN")) == 13) & ((col("weighted_sum_13") % 10) == 0)
)

# Combine checksum validations
weighted_sum = weighted_sum.withColumn(
    "Validity",
    when(col("checksum_valid_10") | col("checksum_valid_13"), "Valid").otherwise("Invalid")
)

# Join back with the original dataset to retrieve titles
books_with_validity = books.join(weighted_sum, on="ISBN", how="inner")

# Filter for invalid ISBNs only
invalid_books = books_with_validity.filter(col("Validity") == "Invalid")

# Select and show ISBNs and titles of invalid books
invalid_books.select("ISBN", "Book_Title", "Book_Author").show(truncate=False)
# Save invalid books to a CSV file
invalid_books.write.csv("invalid_books.csv", header=True, mode="overwrite")
# Stop Spark Session
spark.stop()


+----------+------------------------------------------------------------------------------+-----------------------+
|ISBN      |Book_Title                                                                    |Book_Author            |
+----------+------------------------------------------------------------------------------+-----------------------+
|B0000A2U93|Carmilla                                                                      |Joseph Sheridan Le Fanu|
|B0000633PU|The Story of Aladdin and the Wonderful Lamp                                   |S. Lane Poole          |
|B00007FYKO|Bloodcurdling Tales of Horror and the Macabre: The Best of H. P. Lovecraft    |H. P. Lovecraft        |
|B00008NRHQ|Mystic Rose, The                                                              |Stephen R. Lawhead     |
|B00009ANY9|Cane River                                                                    |Lalita Tademy          |
|B00009APKU|Moby Dick                                                   